In [1]:
import pandas as pd
import numpy as np
import joblib
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import os
import warnings

warnings.filterwarnings("ignore")

DATA_FILE = "train.csv"
MODEL_FILE = "superstore_models.pkl"

def train_and_save():
    print("🚀 Akıllı Model Eğitimi Başlıyor...")
    
    if not os.path.exists(DATA_FILE):
        print(f"❌ HATA: '{DATA_FILE}' bulunamadı!")
        return

    df = pd.read_csv(DATA_FILE)
    df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d/%m/%Y')
    
    # Aylık Gruplama (ME = Month End)
    df_monthly = df.groupby(['Category', pd.Grouper(key='Order Date', freq='ME')])['Sales'].sum().reset_index()
    
    categories = df_monthly['Category'].unique()
    models = {}
    
    for cat in categories:
        print(f"   ⚙️ İşleniyor: {cat}")
        
        # Veriyi Hazırla
        cat_data = df_monthly[df_monthly['Category'] == cat].set_index('Order Date')['Sales']
        cat_data.index.freq = 'ME'
        
        # --- HİBRİT STRATEJİ ---
        # Technology ve Furniture çok dalgalı olduğu için Logaritma kullanıyoruz
        if cat in ['Technology', 'Furniture']:
            print(f"      🔹 Logaritmik Model Seçildi (Volatil Veri)")
            # 1. Log al (Patlamaları törpüle)
            train_data = np.log1p(cat_data)
            
            # 2. Modeli Eğit (Trend yok, Sadece Mevsimsellik)
            model = ExponentialSmoothing(
                train_data,
                seasonal_periods=12,
                trend=None, 
                seasonal='add'
            ).fit(optimized=True)
            
            # Modelin tipini kaydet (Tahmin yaparken Log'u geri çevirmek için lazım)
            models[cat] = {'model': model, 'type': 'log'}
            
        else:
            # Office Supplies daha stabil, Klasik Yöntem
            print(f"      🔸 Klasik Model Seçildi (Stabil Veri)")
            model = ExponentialSmoothing(
                cat_data,
                seasonal_periods=12,
                trend='add',
                seasonal='add'
            ).fit(optimized=True)
            
            models[cat] = {'model': model, 'type': 'linear'}

    # Kaydet
    joblib.dump(models, MODEL_FILE)
    print(f"\n✅ Tüm modeller '{MODEL_FILE}' dosyasına başarıyla kaydedildi.")

if __name__ == "__main__":
    train_and_save()

🚀 Akıllı Model Eğitimi Başlıyor...
   ⚙️ İşleniyor: Furniture
      🔹 Logaritmik Model Seçildi (Volatil Veri)
   ⚙️ İşleniyor: Office Supplies
      🔸 Klasik Model Seçildi (Stabil Veri)
   ⚙️ İşleniyor: Technology
      🔹 Logaritmik Model Seçildi (Volatil Veri)

✅ Tüm modeller 'superstore_models.pkl' dosyasına başarıyla kaydedildi.


In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings

warnings.filterwarnings("ignore")
DATA_FILE = "train.csv"

def evaluate():
    print("📊 YENİ Model Karnesi (Hibrit Motor)...\n")
    df = pd.read_csv(DATA_FILE)
    df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d/%m/%Y')
    df_monthly = df.groupby(['Category', pd.Grouper(key='Order Date', freq='ME')])['Sales'].sum().reset_index()
    
    print(f"{'KATEGORİ':<20} | {'MAPE (Hata)':<15} | {'SKOR (Tahmini)':<15}")
    print("-" * 60)
    
    for cat in df_monthly['Category'].unique():
        cat_data = df_monthly[df_monthly['Category'] == cat].set_index('Order Date')['Sales']
        cat_data.index.freq = 'ME'
        
        train = cat_data.iloc[:-12]
        test = cat_data.iloc[-12:]
        
        # HİBRİT MANTIK (Aynı mantığı burada da uyguluyoruz)
        if cat in ['Technology', 'Furniture']:
            model = ExponentialSmoothing(np.log1p(train), seasonal_periods=12, trend=None, seasonal='add').fit(optimized=True)
            pred = np.expm1(model.forecast(12))
        else:
            model = ExponentialSmoothing(train, seasonal_periods=12, trend='add', seasonal='add').fit(optimized=True)
            pred = model.forecast(12)
            
        mape = np.mean(np.abs((test - pred) / test)) * 100
        score = 100 - mape # Basit Skor Hesabı
        
        print(f"{cat:<20} | %{mape:<14.2f} | {score:.0f}/100")

if __name__ == "__main__":
    evaluate()

📊 YENİ Model Karnesi (Hibrit Motor)...

KATEGORİ             | MAPE (Hata)     | SKOR (Tahmini) 
------------------------------------------------------------
Furniture            | %29.27          | 71/100
Office Supplies      | %31.97          | 68/100
Technology           | %31.14          | 69/100
